In [1]:
import pandas as pd
import numpy as np
import glob

In [2]:
data_list = glob.glob('*csv')
data_list

['final_data.csv',
 'popup_space_추가.csv',
 'Seoul_Festival.csv',
 'serial_code.csv',
 '버스_카페_랜드마크.csv',
 '생활인구_2030_q.csv',
 '서울특별시 지하철 행정동 정보.csv',
 '지하철역(정렬).csv',
 '카드데이터통합본.csv',
 '컨셉_데이터.csv',
 '토지현황비율.csv']

In [3]:
serial_code = pd.read_csv('serial_code.csv')
serial_code.head()

,시리얼,자치구 행정동,상권배후지코드
0,4001,서초구 서초3동,3110296
1,4005,서초구 양재1동,3110314
2,4007,강동구 성내1동,3110404
3,4008,강남구 대치4동,3110331
4,4010,강남구 역삼1동,3110313


---

## 00. 지하철 추가 전처리

In [4]:
sub1 = pd.read_csv('서울특별시 지하철 행정동 정보.csv',encoding='cp949')
sub2 = pd.read_csv('지하철역(정렬).csv')
display(sub1.head())
display(sub2.head())

,역번호,역사명,노선번호,노선명,역위도,역경도,역사도로명주소,시군구,행정동
0,219,삼성(무역센터),S1102,2호선,NaN,NaN,서울특별시 강남구 테헤란로 지하538(삼성동),강남구,삼성1동
1,220,선릉,S1102,2호선,NaN,NaN,서울특별시 강남구 테헤란로 지하340(삼성동),강남구,삼성2동
2,221,역삼,S1102,2호선,NaN,NaN,서울특별시 강남구 테헤란로 지하156(역삼동),강남구,역삼1동
3,222,강남,S1102,2호선,NaN,NaN,서울특별시 강남구 강남대로 지하396(역삼동),강남구,역삼1동
4,336,압구정,I1103,3호선,NaN,NaN,서울특별시 강남구 압구정로 지하172(신사동),강남구,신사동


,역번호,호선,역명,지번주소,주소
0,335,3,대치,서울특별시 강남구 대치동 317-3 대치역(3호선),강남구 대치동
1,336,3,학여울,서울특별시 강남구 대치동 514-3 학여울역(3호선),강남구 대치동
2,327,3,신사,서울특별시 강남구 신사동 667 신사역(3호선),강남구 신사동
3,326,3,압구정,서울특별시 강남구 신사동 668 압구정역(3호선),강남구 신사동
4,222,2,강남,서울특별시 강남구 역삼동 858 강남역(2호선),강남구 역삼동


In [6]:
df = serial_code.copy()
df['행정동'] = 0
df['자치구'] = 0
for i in range(len(df)):
    df['자치구'][i] = df['자치구 행정동'][i].split(' ')[0]
    df['행정동'][i] = df['자치구 행정동'][i].split(' ')[1]
    
sub1_df = sub1.groupby(['행정동'],as_index=False).count()[['행정동','역번호']]
sub1_df.head()

df = pd.merge(df,sub1_df,on='행정동',how='left')
df.fillna(0, inplace=True)
df.head()

C:\Users\user\AppData\Local\Temp\ipykernel_16988\3479915892.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['자치구'][i] = df['자치구 행정동'][i].split(' ')[0]
C:\Users\user\AppData\Local\Temp\ipykernel_16988\3479915892.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['행정동'][i] = df['자치구 행정동'][i].split(' ')[1]


,시리얼,자치구 행정동,상권배후지코드,행정동,자치구,역번호
0,4001,서초구 서초3동,3110296,서초3동,서초구,1.0
1,4005,서초구 양재1동,3110314,양재1동,서초구,2.0
2,4007,강동구 성내1동,3110404,성내1동,강동구,1.0
3,4008,강남구 대치4동,3110331,대치4동,강남구,0.0
4,4010,강남구 역삼1동,3110313,역삼1동,강남구,5.0


In [7]:
sub2_df = sub2.groupby(['주소'],as_index=False).count()[['주소','역번호']]
sub2_df.head()

,주소,역번호
0,강남구 대치동,2
1,강남구 신사동,2
2,강남구 역삼동,2
3,강동구 성내동,1
4,강동구 암사동,1


In [8]:
for i in range(len(df)):
    for j in range(len(sub2_df)):
        if df['자치구 행정동'][i] == sub2_df['주소'][j]:
            df['자치구 행정동'][i] == max(df['자치구 행정동'][i],sub2_df['주소'][j])

In [9]:
df_sub = df.copy()
df_sub.head()

,시리얼,자치구 행정동,상권배후지코드,행정동,자치구,역번호
0,4001,서초구 서초3동,3110296,서초3동,서초구,1.0
1,4005,서초구 양재1동,3110314,양재1동,서초구,2.0
2,4007,강동구 성내1동,3110404,성내1동,강동구,1.0
3,4008,강남구 대치4동,3110331,대치4동,강남구,0.0
4,4010,강남구 역삼1동,3110313,역삼1동,강남구,5.0


---
## 00. 지하철 + 생활인구

In [10]:
data_2030 = pd.read_csv('생활인구_2030_q.csv')
data_2030.head()

,상권배후지코드,2030 생활인구 1월,2030 생활인구 2월,2030 생활인구 3월,2030 생활인구 4월,2030 생활인구 5월,2030 생활인구 6월,2030 생활인구 7월,2030 생활인구 8월,2030 생활인구 9월,2030 생활인구 10월,2030 생활인구 11월,2030 생활인구 12월
0,3110016,1968.710085,1707.387962,1488.047872,2190.369165,2601.389877,3211.967439,3059.606969,3179.190864,4175.201403,5169.939932,4613.710816,4751.886357
1,3110017,271.446135,258.268952,236.705853,431.200278,543.728595,643.482581,531.097380,616.701177,1052.335976,1269.849809,880.164107,770.341054
2,3110020,2521.198580,2323.667554,1451.254848,2358.417561,1286.896082,2096.480583,1784.735706,2434.005840,3116.525818,2553.311115,2408.015630,4114.080364
3,3110027,592.263815,598.890069,545.001443,701.056675,728.606188,666.607791,571.303207,595.661748,834.294538,817.309517,709.547826,525.465404
4,3110033,526.664419,528.976730,539.327336,558.353659,520.634747,641.048678,588.496830,535.853204,703.897657,769.522561,735.846257,378.849379


In [11]:
final_data = pd.merge(df_sub,data_2030,on='상권배후지코드')
final_data.head()

,시리얼,자치구 행정동,상권배후지코드,행정동,자치구,역번호,2030 생활인구 1월,2030 생활인구 2월,2030 생활인구 3월,2030 생활인구 4월,2030 생활인구 5월,2030 생활인구 6월,2030 생활인구 7월,2030 생활인구 8월,2030 생활인구 9월,2030 생활인구 10월,2030 생활인구 11월,2030 생활인구 12월
0,4001,서초구 서초3동,3110296,서초3동,서초구,1.0,790.407614,755.658301,730.997599,305.022823,789.127508,853.268762,798.409268,884.891582,1005.424667,927.637480,775.089360,949.590781
1,4005,서초구 양재1동,3110314,양재1동,서초구,2.0,1768.003305,1719.058434,1668.646221,1918.467468,2347.575212,2216.537271,2067.146009,2054.968060,2210.967377,1937.904518,1819.651640,1592.384552
2,4007,강동구 성내1동,3110404,성내1동,강동구,1.0,397.657250,361.319285,371.392273,559.414498,534.513210,527.639933,480.493531,448.525969,620.121572,577.333674,533.262627,364.741483
3,4008,강남구 대치4동,3110331,대치4동,강남구,0.0,1500.745052,1284.206000,1171.031123,1400.008013,1265.143119,155.508865,134.310798,114.386180,525.558523,930.106952,1561.813248,1380.781791
4,4010,강남구 역삼1동,3110313,역삼1동,강남구,5.0,8016.949756,7402.491122,6316.755146,7976.575585,8787.598058,9583.673044,9886.709313,8332.615669,9474.083534,9896.063089,9788.474586,8609.679205


---
## 00. 지하철 + 생활인구 + 버스,카페,랜드마크

In [12]:
df_cafe = pd.read_csv('버스_카페_랜드마크.csv',encoding='cp949',usecols=['ADM_NM','버스정류장','랜드마크','카페'])
display(df_cafe.head())
display(final_data.head(3))

,ADM_NM,버스정류장,랜드마크,카페
0,대치4동,14,3,123
1,역삼1동,50,16,434
2,압구정동,25,15,132
3,성내1동,21,2,71
4,서초3동,75,28,194


,시리얼,자치구 행정동,상권배후지코드,행정동,자치구,역번호,2030 생활인구 1월,2030 생활인구 2월,2030 생활인구 3월,2030 생활인구 4월,2030 생활인구 5월,2030 생활인구 6월,2030 생활인구 7월,2030 생활인구 8월,2030 생활인구 9월,2030 생활인구 10월,2030 생활인구 11월,2030 생활인구 12월
0,4001,서초구 서초3동,3110296,서초3동,서초구,1.0,790.407614,755.658301,730.997599,305.022823,789.127508,853.268762,798.409268,884.891582,1005.424667,927.637480,775.089360,949.590781
1,4005,서초구 양재1동,3110314,양재1동,서초구,2.0,1768.003305,1719.058434,1668.646221,1918.467468,2347.575212,2216.537271,2067.146009,2054.968060,2210.967377,1937.904518,1819.651640,1592.384552
2,4007,강동구 성내1동,3110404,성내1동,강동구,1.0,397.657250,361.319285,371.392273,559.414498,534.513210,527.639933,480.493531,448.525969,620.121572,577.333674,533.262627,364.741483


In [13]:
final_data = pd.merge(final_data,df_cafe,left_on='행정동',right_on='ADM_NM',how='left')
print(final_data.shape)
display(final_data.head(3))

(26, 22)


,시리얼,자치구 행정동,상권배후지코드,행정동,자치구,역번호,2030 생활인구 1월,2030 생활인구 2월,2030 생활인구 3월,2030 생활인구 4월,...,2030 생활인구 7월,2030 생활인구 8월,2030 생활인구 9월,2030 생활인구 10월,2030 생활인구 11월,2030 생활인구 12월,ADM_NM,버스정류장,랜드마크,카페
0,4001,서초구 서초3동,3110296,서초3동,서초구,1.0,790.407614,755.658301,730.997599,305.022823,...,798.409268,884.891582,1005.424667,927.637480,775.089360,949.590781,서초3동,75,28,194
1,4005,서초구 양재1동,3110314,양재1동,서초구,2.0,1768.003305,1719.058434,1668.646221,1918.467468,...,2067.146009,2054.968060,2210.967377,1937.904518,1819.651640,1592.384552,양재1동,73,4,114
2,4007,강동구 성내1동,3110404,성내1동,강동구,1.0,397.657250,361.319285,371.392273,559.414498,...,480.493531,448.525969,620.121572,577.333674,533.262627,364.741483,성내1동,21,2,71


---
## 00. 지하철 + 생활인구 + 버스,카페,랜드마크 + 토지

In [14]:
df_land = pd.read_csv('토지현황비율.csv')
df_land.head()

,동별(2),자연 비율(%),공장용지 등 비율(%),도로 등 비율(%),학교용지 등 비율(%)
0,소계,22.745302,0.575126,15.110393,8.783684
1,종로구,39.138858,0.024985,10.030884,5.214597
2,중구,12.908926,0.120549,22.704554,9.317013
3,용산구,7.328208,0.048372,17.731037,3.137599
4,성동구,1.530381,2.945841,17.397019,11.704387


In [15]:
final_data = pd.merge(final_data,df_land,left_on='자치구',right_on='동별(2)',how='left')
final_data.head(3)

,시리얼,자치구 행정동,상권배후지코드,행정동,자치구,역번호,2030 생활인구 1월,2030 생활인구 2월,2030 생활인구 3월,2030 생활인구 4월,...,2030 생활인구 12월,ADM_NM,버스정류장,랜드마크,카페,동별(2),자연 비율(%),공장용지 등 비율(%),도로 등 비율(%),학교용지 등 비율(%)
0,4001,서초구 서초3동,3110296,서초3동,서초구,1.0,790.407614,755.658301,730.997599,305.022823,...,949.590781,서초3동,75,28,194,서초구,37.949724,0.122745,13.883397,4.352688
1,4005,서초구 양재1동,3110314,양재1동,서초구,2.0,1768.003305,1719.058434,1668.646221,1918.467468,...,1592.384552,양재1동,73,4,114,서초구,37.949724,0.122745,13.883397,4.352688
2,4007,강동구 성내1동,3110404,성내1동,강동구,1.0,397.657250,361.319285,371.392273,559.414498,...,364.741483,성내1동,21,2,71,강동구,11.513797,0.130624,18.180446,9.504325


---
## 00. 지하철 + 생활인구 + 버스,카페,랜드마크 + 토지 + 축제 (이벤트)

In [17]:
df_event = pd.read_csv('Seoul_Festival.csv',encoding='cp949')
df_event.head(3)

,분류,자치구,행정동,공연/행사명,날짜/시간,장소,위도(X좌표),경도(Y좌표),시작일,종료일,...,2023년9월행사,2023년8월행사,2023년7월행사,2023년6월행사,2023년5월행사,2023년4월행사,2023년3월행사,2023년2월행사,2023년1월행사,2022년12월행사
0,콘서트,강남구,신사동,"[2023 카즈미 타테이시 트리오 내한공연] 지브리, 재즈를 만나다(서울 강남)",2023-11-06~2023-11-06,광림아트센터 장천홀,37.524356,127.025985,11/6/2023,11/6/2023,...,0,0,0,0,0,0,0,0,0,0
1,뮤지컬/오페라,강남구,역삼1동,[8월 키즈예술공연 라이브 국악 뮤지컬] 꼬마 강치전,2023-08-19~2023-08-19,강남씨어터,37.495373,127.033284,8/19/2023,8/19/2023,...,0,1,0,0,0,0,0,0,0,0
2,전시/미술,강남구,청담동,[Beyond the Atolls],2023-05-02~2023-06-10,호리아트스페이스,37.524799,127.044611,5/2/2023,6/10/2023,...,0,0,0,1,1,0,0,0,0,0


In [18]:
df_event.columns

Index(['분류', '자치구', '행정동', '공연/행사명', '날짜/시간', '장소', '위도(X좌표)', '경도(Y좌표)',
       '시작일', '종료일', '2023년11월행사', '2023년12월행사', '2024년1월행사', '2024년3월행사',
       '2024년2월행사', '2023년10월행사', '2023년9월행사', '2023년8월행사', '2023년7월행사',
       '2023년6월행사', '2023년5월행사', '2023년4월행사', '2023년3월행사', '2023년2월행사',
       '2023년1월행사', '2022년12월행사'],
      dtype='object')

In [19]:
df_event = df_event[['자치구','행정동','2024년3월행사','2024년2월행사','2024년1월행사',
        '2023년12월행사','2023년11월행사', '2023년10월행사', '2023년9월행사', '2023년8월행사', '2023년7월행사',
       '2023년6월행사', '2023년5월행사', '2023년4월행사', '2023년3월행사', '2023년2월행사', '2023년1월행사', 
          '2022년12월행사']]
display(df_event.head())
display(df_event.tail())

,자치구,행정동,2024년3월행사,2024년2월행사,2024년1월행사,2023년12월행사,2023년11월행사,2023년10월행사,2023년9월행사,2023년8월행사,2023년7월행사,2023년6월행사,2023년5월행사,2023년4월행사,2023년3월행사,2023년2월행사,2023년1월행사,2022년12월행사
0,강남구,신사동,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0
1,강남구,역삼1동,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0
2,강남구,청담동,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0
3,강남구,역삼1동,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0
4,강남구,신사동,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0


,자치구,행정동,2024년3월행사,2024년2월행사,2024년1월행사,2023년12월행사,2023년11월행사,2023년10월행사,2023년9월행사,2023년8월행사,2023년7월행사,2023년6월행사,2023년5월행사,2023년4월행사,2023년3월행사,2023년2월행사,2023년1월행사,2022년12월행사
3427,중랑구,신내2동,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0
3428,중랑구,신내2동,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0
3429,중랑구,신내2동,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0
3430,중랑구,상봉1동,0,0,0,0,0,0,0,0,0,1,1,1,0,0,0,0
3431,중랑구,면목4동,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0


In [20]:
df_event_gr = df_event.groupby(['행정동'],as_index=False).sum()

final_data = pd.merge(final_data,df_event_gr,on='행정동',how='left',suffixes=['','.'])
final_data.fillna(0,inplace=True)
final_data.head(5)

C:\Users\user\AppData\Local\Temp\ipykernel_16988\242285107.py:1: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  df_event_gr = df_event.groupby(['행정동'],as_index=False).sum()


,시리얼,자치구 행정동,상권배후지코드,행정동,자치구,역번호,2030 생활인구 1월,2030 생활인구 2월,2030 생활인구 3월,2030 생활인구 4월,...,2023년9월행사,2023년8월행사,2023년7월행사,2023년6월행사,2023년5월행사,2023년4월행사,2023년3월행사,2023년2월행사,2023년1월행사,2022년12월행사
0,4001,서초구 서초3동,3110296,서초3동,서초구,1.0,790.407614,755.658301,730.997599,305.022823,...,8.0,5.0,4.0,4.0,5.0,4.0,7.0,5.0,3.0,0.0
1,4005,서초구 양재1동,3110314,양재1동,서초구,2.0,1768.003305,1719.058434,1668.646221,1918.467468,...,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0
2,4007,강동구 성내1동,3110404,성내1동,강동구,1.0,397.657250,361.319285,371.392273,559.414498,...,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
3,4008,강남구 대치4동,3110331,대치4동,강남구,0.0,1500.745052,1284.206000,1171.031123,1400.008013,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,4010,강남구 역삼1동,3110313,역삼1동,강남구,5.0,8016.949756,7402.491122,6316.755146,7976.575585,...,0.0,3.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0


---
## 00. 지하철 + 생활인구 + 버스,카페,랜드마크 + 토지 + 축제 (이벤트) + 카드

In [21]:
card = pd.read_csv('카드데이터통합본.csv')
card.head()

,지역,가전/가구 금액비율_4분기,교육용품 금액비율_4분기,기타요식 금액비율_4분기,기타유통 금액비율_4분기,기타의료 금액비율_4분기,미용서비스 금액비율_4분기,백화점 금액비율_4분기,병원 금액비율_4분기,서비스 금액비율_4분기,...,자동차서비스/용품 건수비율_3분기,자동차판매 건수비율_3분기,제과/커피/패스트푸드 건수비율_3분기,주유 건수비율_3분기,패션/잡화 건수비율_3분기,편의점 건수비율_3분기,학원 건수비율_3분기,한식 건수비율_3분기,할인점/슈퍼마켓 건수비율_3분기,화장품 건수비율_3분기
0,강남구 개포1동,0.000000,0.000000,0.000004,0.000000,0.000000,0.000010,0.0,0.000009,0.000089,...,0.000004,0.000000,0.000000,0.000000,0.000000,0.000111,0.000056,0.000005,0.000142,0.000000
1,강남구 개포2동,0.000002,0.000519,0.000530,0.000000,0.000311,0.001059,0.0,0.000399,0.000509,...,0.000296,0.000000,0.001307,0.000000,0.000117,0.000840,0.002748,0.000871,0.000981,0.000268
2,강남구 개포4동,0.002850,0.001425,0.000824,0.000057,0.000998,0.000840,0.0,0.000576,0.000801,...,0.000814,0.001180,0.000830,0.000631,0.000300,0.001417,0.000828,0.000674,0.001323,0.000099
3,강남구 논현1동,0.020151,0.008603,0.008604,0.000000,0.002407,0.022318,0.0,0.039587,0.016064,...,0.004029,0.004130,0.007119,0.002964,0.013497,0.008650,0.004630,0.009013,0.002434,0.002312
4,강남구 논현2동,0.015127,0.041660,0.006971,0.000140,0.008285,0.011394,0.0,0.012873,0.050507,...,0.003824,0.014159,0.010008,0.004946,0.004190,0.007370,0.010257,0.009912,0.002926,0.005668


In [22]:
final_data = pd.merge(final_data,card,left_on='자치구 행정동',right_on='지역')
final_data.head(3)

,시리얼,자치구 행정동,상권배후지코드,행정동,자치구,역번호,2030 생활인구 1월,2030 생활인구 2월,2030 생활인구 3월,2030 생활인구 4월,...,자동차서비스/용품 건수비율_3분기,자동차판매 건수비율_3분기,제과/커피/패스트푸드 건수비율_3분기,주유 건수비율_3분기,패션/잡화 건수비율_3분기,편의점 건수비율_3분기,학원 건수비율_3분기,한식 건수비율_3분기,할인점/슈퍼마켓 건수비율_3분기,화장품 건수비율_3분기
0,4001,서초구 서초3동,3110296,서초3동,서초구,1.0,790.407614,755.658301,730.997599,305.022823,...,0.004375,0.011013,0.009462,0.004328,0.001528,0.008270,0.008394,0.010697,0.004508,0.001354
1,4005,서초구 양재1동,3110314,양재1동,서초구,2.0,1768.003305,1719.058434,1668.646221,1918.467468,...,0.002430,0.006883,0.004313,0.004019,0.004274,0.004180,0.009787,0.004984,0.002738,0.000761
2,4007,강동구 성내1동,3110404,성내1동,강동구,1.0,397.657250,361.319285,371.392273,559.414498,...,0.000390,0.017896,0.002093,0.001423,0.000977,0.001483,0.003576,0.001458,0.001613,0.000860


In [23]:
final_data['행정동'].unique()

array(['서초3동', '양재1동', '성내1동', '대치4동', '역삼1동', '압구정동', '신당동', '명동', '회현동',
       '녹번동', '대조동', '방화3동', '방화1동', '창1동', '홍제3동', '천호1동', '낙성대동',
       '신정4동', '가회동', '이화동', '성수1가2동', '성수1가1동', '신사동', '화양동', '구의1동',
       '천연동'], dtype=object)

In [24]:
len(final_data['행정동'].unique()) , len(final_data['자치구 행정동'].unique())

(26, 26)

---
## 00. 지하철 + 생활인구 + 버스,카페,랜드마크 + 토지 + 축제 (이벤트) + 카드 + 팝업데이터

In [25]:
space = pd.read_csv('popup_space_추가.csv')
space.head()

,name,add,real_add,행정동,price,총층수,1층포함여부,평수(㎡),대관단위(시간),총평수,수용인원수,시간당대관료,평당가격
0,홍대입구역 홍대 정문 팝업 통건물 by SHARE.IT,서울 마포구,서울 마포구 와우산로 101,NaN,10000000,6,1,697.0,24,4182.0,1394,416666.66670,99.633349
1,성수 팝업 서울숲 위팝 1st by SHARE.IT,서울 성동구,서울 성동구 서울숲4길 20,성수1가2동,800000,1,1,48.0,23,48.0,16,34782.60870,724.637681
2,성수 팝업 서울숲 위팝 2nd by SHARE.IT,서울 성동구,서울 성동구 서울숲4길 26 1층,성수1가2동,900000,1,1,59.4,23,59.4,20,39130.43478,658.761528
3,성수 팝업 연무장 온더무브 by SHARE.IT,서울 성동구,서울 성동구 연무장길 13 1층,성수2가3동,900000,1,1,36.0,23,36.0,12,39130.43478,1086.956522
4,성수 서울숲 팝업 건물 통대관 by SHARE.IT,서울 성동구,서울 성동구 서울숲4길 15-1 통건물,성수1가2동,6000000,6,0,697.0,23,4182.0,1394,260869.56520,62.379140


In [26]:
size = []

for i in range(len(space)):
    if space['총평수'][i] <= space['총평수'].quantile(0.25):
        size.append('소')
    elif space['총평수'][i] <= space['총평수'].quantile(0.5):
        size.append('중') 
    else:
        size.append('대')
        
space['공간크기'] = size
space

,name,add,real_add,행정동,price,총층수,1층포함여부,평수(㎡),대관단위(시간),총평수,수용인원수,시간당대관료,평당가격,공간크기
0,홍대입구역 홍대 정문 팝업 통건물 by SHARE.IT,서울 마포구,서울 마포구 와우산로 101,NaN,10000000,6,1,697.0,24,4182.0,1394,4.166667e+05,99.633349,대
1,성수 팝업 서울숲 위팝 1st by SHARE.IT,서울 성동구,서울 성동구 서울숲4길 20,성수1가2동,800000,1,1,48.0,23,48.0,16,3.478261e+04,724.637681,소
2,성수 팝업 서울숲 위팝 2nd by SHARE.IT,서울 성동구,서울 성동구 서울숲4길 26 1층,성수1가2동,900000,1,1,59.4,23,59.4,20,3.913043e+04,658.761528,소
3,성수 팝업 연무장 온더무브 by SHARE.IT,서울 성동구,서울 성동구 연무장길 13 1층,성수2가3동,900000,1,1,36.0,23,36.0,12,3.913043e+04,1086.956522,소
4,성수 서울숲 팝업 건물 통대관 by SHARE.IT,서울 성동구,서울 성동구 서울숲4길 15-1 통건물,성수1가2동,6000000,6,0,697.0,23,4182.0,1394,2.608696e+05,62.379140,대
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
371,명동빌딩 맞은편 대관 공간B(2층),서울 중구,서울 중구 명동길 74,명동,0,1,0,687.0,1,687.0,229,1.200000e+06,1746.724891,대
372,명동빌딩 맞은편 대관 공간C(3층),서울 중구,서울 중구 명동길 75,명동,0,1,0,477.0,1,477.0,159,1.200000e+06,2515.723270,대
373,명동빌딩 맞은편 대관 공간D(4층),서울 중구,서울 중구 명동길 76,명동,0,1,0,592.0,1,592.0,197,1.200000e+06,2027.027027,대
374,갤러리 지오타 (한옥),서울 가회동,서울 종로구 북촌로 63-9,가회동,250000,1,1,66.0,10,66.0,22,2.500000e+04,378.787879,소


In [27]:
space['행정동'].unique()

array([nan, '성수1가2동', '성수2가3동', '성수1가1동', '을지로동', '성수2가1동', '청담동', '압구정동',
       '한남동', '삼성1동', '금호4가동', '신사동', '화곡6동', '한강로동', '종로1234가동', '송정동',
       '이태원1동', '역삼1동', '황학동', '사직동', '논현2동', '혜화동', '연희동', '소공동', '논현1동',
       '이태원2동', '삼청동', '회현동', '명동', '반포동', '능동', '잠원동', '창신1동', '군자동',
       '신촌동', '양재1동', '삼성2동', '화양동', '광희동', '후암동', '서초3동', '대치4동', '청구동',
       '가회동', '신당동'], dtype=object)

In [28]:
final_data = pd.merge(space,final_data,on='행정동',how='outer')
print(len(final_data))
final_data.head()

389


,name,add,real_add,행정동,price,총층수,1층포함여부,평수(㎡),대관단위(시간),총평수,...,자동차서비스/용품 건수비율_3분기,자동차판매 건수비율_3분기,제과/커피/패스트푸드 건수비율_3분기,주유 건수비율_3분기,패션/잡화 건수비율_3분기,편의점 건수비율_3분기,학원 건수비율_3분기,한식 건수비율_3분기,할인점/슈퍼마켓 건수비율_3분기,화장품 건수비율_3분기
0,홍대입구역 홍대 정문 팝업 통건물 by SHARE.IT,서울 마포구,서울 마포구 와우산로 101,NaN,10000000.0,6.0,1.0,697.0,24.0,4182.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,DRC 홍대 팝업 통건물 by Share.IT,서울 마포구,서울 마포구 와우산로 82,NaN,5000000.0,4.0,1.0,295.5,23.0,1182.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,연남방앗간,서울 마포구,서울 마포구 동교로29길 34,NaN,3000000.0,3.0,1.0,101.0,23.0,303.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,러빈허,서울 마포구,서울 마포구 신촌로6길 22 동교동 177-12,NaN,2000000.0,1.0,1.0,58.7,24.0,58.7,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,커피덕 홍대,서울 마포구,서울 마포구 월드컵북로2길 37,NaN,5000000.0,1.0,1.0,330.0,24.0,330.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [29]:
final_data['행정동'].unique()

array([nan, '성수1가2동', '성수2가3동', '성수1가1동', '을지로동', '성수2가1동', '청담동', '압구정동',
       '한남동', '삼성1동', '금호4가동', '신사동', '화곡6동', '한강로동', '종로1234가동', '송정동',
       '이태원1동', '역삼1동', '황학동', '사직동', '논현2동', '혜화동', '연희동', '소공동', '논현1동',
       '이태원2동', '삼청동', '회현동', '명동', '반포동', '능동', '잠원동', '창신1동', '군자동',
       '신촌동', '양재1동', '삼성2동', '화양동', '광희동', '후암동', '서초3동', '대치4동', '청구동',
       '가회동', '신당동', '성내1동', '녹번동', '대조동', '방화3동', '방화1동', '창1동', '홍제3동',
       '천호1동', '낙성대동', '신정4동', '이화동', '구의1동', '천연동'], dtype=object)

---
## 00. 지하철 + 생활인구 + 버스,카페,랜드마크 + 토지 + 축제 (이벤트) + 카드 + 팝업데이터 + 컨셉데이터

In [30]:
df_concept = pd.read_csv('컨셉_데이터.csv')
df_concept

,자치구 행정동,골목_컨셉,갤러리_컨셉,음악_컨셉,행정동,한옥 개수,예술_컨셉
0,성동구 성수1가2동,2.0,0.0,0.0,성수1가2동,0.0,0.0
1,성동구 성수1가1동,2.0,0.0,0.0,성수1가1동,0.0,1.0
2,강남구 압구정동,2.0,0.0,0.0,압구정동,0.0,0.0
3,강남구 신사동,4.0,0.0,11.0,신사동,0.0,0.0
4,강남구 역삼1동,0.0,0.0,0.0,역삼1동,0.0,0.0
...,...,...,...,...,...,...,...
341,0,0.0,0.0,0.0,천호2동,0.0,1.0
342,0,0.0,0.0,0.0,필동,0.0,4.0
343,0,0.0,0.0,0.0,행당1동,0.0,2.0
344,0,0.0,0.0,0.0,홍은1동,0.0,1.0


In [31]:
final_data = pd.merge(final_data,df_concept[['행정동','골목_컨셉','갤러리_컨셉','음악_컨셉','예술_컨셉','한옥 개수']],on='행정동',how='outer')
print(len(final_data))
final_data.head()

709


,name,add,real_add,행정동,price,총층수,1층포함여부,평수(㎡),대관단위(시간),총평수,...,편의점 건수비율_3분기,학원 건수비율_3분기,한식 건수비율_3분기,할인점/슈퍼마켓 건수비율_3분기,화장품 건수비율_3분기,골목_컨셉,갤러리_컨셉,음악_컨셉,예술_컨셉,한옥 개수
0,홍대입구역 홍대 정문 팝업 통건물 by SHARE.IT,서울 마포구,서울 마포구 와우산로 101,NaN,10000000.0,6.0,1.0,697.0,24.0,4182.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,DRC 홍대 팝업 통건물 by Share.IT,서울 마포구,서울 마포구 와우산로 82,NaN,5000000.0,4.0,1.0,295.5,23.0,1182.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,연남방앗간,서울 마포구,서울 마포구 동교로29길 34,NaN,3000000.0,3.0,1.0,101.0,23.0,303.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,러빈허,서울 마포구,서울 마포구 신촌로6길 22 동교동 177-12,NaN,2000000.0,1.0,1.0,58.7,24.0,58.7,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,커피덕 홍대,서울 마포구,서울 마포구 월드컵북로2길 37,NaN,5000000.0,1.0,1.0,330.0,24.0,330.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


---

## 00. 중복 칼럼 삭제

In [32]:
final_data.columns[:100]

Index(['name', 'add', 'real_add', '행정동', 'price', '총층수', '1층포함여부', '평수(㎡)',
       '대관단위(시간)', '총평수', '수용인원수', '시간당대관료', '평당가격', '공간크기', '시리얼', '자치구 행정동',
       '상권배후지코드', '자치구', '역번호', '2030 생활인구 1월', '2030 생활인구 2월', '2030 생활인구 3월',
       '2030 생활인구 4월', '2030 생활인구 5월', '2030 생활인구 6월', '2030 생활인구 7월',
       '2030 생활인구 8월', '2030 생활인구 9월', '2030 생활인구 10월', '2030 생활인구 11월',
       '2030 생활인구 12월', 'ADM_NM', '버스정류장', '랜드마크', '카페', '동별(2)', '자연 비율(%)',
       '공장용지 등 비율(%)', '도로 등 비율(%)', '학교용지 등 비율(%)', '2024년3월행사', '2024년2월행사',
       '2024년1월행사', '2023년12월행사', '2023년11월행사', '2023년10월행사', '2023년9월행사',
       '2023년8월행사', '2023년7월행사', '2023년6월행사', '2023년5월행사', '2023년4월행사',
       '2023년3월행사', '2023년2월행사', '2023년1월행사', '2022년12월행사', '지역',
       '가전/가구 금액비율_4분기', '교육용품 금액비율_4분기', '기타요식 금액비율_4분기', '기타유통 금액비율_4분기',
       '기타의료 금액비율_4분기', '미용서비스 금액비율_4분기', '백화점 금액비율_4분기', '병원 금액비율_4분기',
       '서비스 금액비율_4분기', '숙박 금액비율_4분기', '스포츠/문화/레저 금액비율_4분기',
       '스포츠/문화/레저용품 금액비율_4분기', '약국 금액비율_4분

In [33]:
final_data.columns[100:200]

Index(['여행 건수비율_4분기', '유아교육 건수비율_4분기', '유흥 건수비율_4분기', '음/식료품 건수비율_4분기',
       '의복/의류 건수비율_4분기', '인테리어 건수비율_4분기', '일식/중식/양식 건수비율_4분기',
       '자동차서비스/용품 건수비율_4분기', '자동차판매 건수비율_4분기', '제과/커피/패스트푸드 건수비율_4분기',
       '주유 건수비율_4분기', '패션/잡화 건수비율_4분기', '편의점 건수비율_4분기', '학원 건수비율_4분기',
       '한식 건수비율_4분기', '할인점/슈퍼마켓 건수비율_4분기', '화장품 건수비율_4분기', '가전/가구 금액비율_1분기',
       '교육용품 금액비율_1분기', '기타요식 금액비율_1분기', '기타유통 금액비율_1분기', '기타의료 금액비율_1분기',
       '미용서비스 금액비율_1분기', '백화점 금액비율_1분기', '병원 금액비율_1분기', '서비스 금액비율_1분기',
       '숙박 금액비율_1분기', '스포츠/문화/레저 금액비율_1분기', '스포츠/문화/레저용품 금액비율_1분기',
       '약국 금액비율_1분기', '여행 금액비율_1분기', '유아교육 금액비율_1분기', '유흥 금액비율_1분기',
       '음/식료품 금액비율_1분기', '의복/의류 금액비율_1분기', '인테리어 금액비율_1분기',
       '일식/중식/양식 금액비율_1분기', '자동차서비스/용품 금액비율_1분기', '자동차판매 금액비율_1분기',
       '제과/커피/패스트푸드 금액비율_1분기', '주유 금액비율_1분기', '패션/잡화 금액비율_1분기', '편의점 금액비율_1분기',
       '학원 금액비율_1분기', '한식 금액비율_1분기', '할인점/슈퍼마켓 금액비율_1분기', '화장품 금액비율_1분기',
       '가전/가구 건수비율_1분기', '교육용품 건수비율_1분기', '기타요식 건수비율_1분기', '기타유통 건수비율_1분기',
    

In [34]:
final_data.columns[200:]

Index(['주유 금액비율_2분기', '패션/잡화 금액비율_2분기', '편의점 금액비율_2분기', '학원 금액비율_2분기',
       '한식 금액비율_2분기', '할인점/슈퍼마켓 금액비율_2분기', '화장품 금액비율_2분기', '가전/가구 건수비율_2분기',
       '교육용품 건수비율_2분기', '기타요식 건수비율_2분기',
       ...
       '편의점 건수비율_3분기', '학원 건수비율_3분기', '한식 건수비율_3분기', '할인점/슈퍼마켓 건수비율_3분기',
       '화장품 건수비율_3분기', '골목_컨셉', '갤러리_컨셉', '음악_컨셉', '예술_컨셉', '한옥 개수'],
      dtype='object', length=102)

In [35]:
drop_col = ['ADM_NM','동별(2)','지역']
final_data.drop(drop_col,axis=1,inplace=True)
final_data.head()

,name,add,real_add,행정동,price,총층수,1층포함여부,평수(㎡),대관단위(시간),총평수,...,편의점 건수비율_3분기,학원 건수비율_3분기,한식 건수비율_3분기,할인점/슈퍼마켓 건수비율_3분기,화장품 건수비율_3분기,골목_컨셉,갤러리_컨셉,음악_컨셉,예술_컨셉,한옥 개수
0,홍대입구역 홍대 정문 팝업 통건물 by SHARE.IT,서울 마포구,서울 마포구 와우산로 101,NaN,10000000.0,6.0,1.0,697.0,24.0,4182.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,DRC 홍대 팝업 통건물 by Share.IT,서울 마포구,서울 마포구 와우산로 82,NaN,5000000.0,4.0,1.0,295.5,23.0,1182.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,연남방앗간,서울 마포구,서울 마포구 동교로29길 34,NaN,3000000.0,3.0,1.0,101.0,23.0,303.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,러빈허,서울 마포구,서울 마포구 신촌로6길 22 동교동 177-12,NaN,2000000.0,1.0,1.0,58.7,24.0,58.7,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,커피덕 홍대,서울 마포구,서울 마포구 월드컵북로2길 37,NaN,5000000.0,1.0,1.0,330.0,24.0,330.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [36]:
final_data = final_data[final_data['name'].notnull()] # 이름이 존재하는 행
print('이름 존재:',final_data.shape)

이름 존재: (376, 299)


In [37]:
final_data = final_data[final_data['행정동'].notnull()] # 행정동이 존재하는 행
print('이름, 행정동 존재:',final_data.shape)

이름, 행정동 존재: (288, 299)


In [38]:
final_data = final_data[final_data['화장품 건수비율_3분기'].notnull()] # 카드데이터가 존재하는 행
print('이름, 행정동, 카드 존재:',final_data.shape)

이름, 행정동, 카드 존재: (105, 299)


In [39]:
final_data.reset_index(drop=True,inplace=True)

In [40]:
final_data.to_csv('final_data.csv',index=False)
pd.read_csv('final_data.csv')

,name,add,real_add,행정동,price,총층수,1층포함여부,평수(㎡),대관단위(시간),총평수,...,편의점 건수비율_3분기,학원 건수비율_3분기,한식 건수비율_3분기,할인점/슈퍼마켓 건수비율_3분기,화장품 건수비율_3분기,골목_컨셉,갤러리_컨셉,음악_컨셉,예술_컨셉,한옥 개수
0,성수 팝업 서울숲 위팝 1st by SHARE.IT,서울 성동구,서울 성동구 서울숲4길 20,성수1가2동,800000.0,1.0,1.0,48.0,23.0,48.0,...,0.003625,0.001205,0.005915,0.001986,0.006091,2.0,0.0,0.0,0.0,0.0
1,성수 팝업 서울숲 위팝 2nd by SHARE.IT,서울 성동구,서울 성동구 서울숲4길 26 1층,성수1가2동,900000.0,1.0,1.0,59.4,23.0,59.4,...,0.003625,0.001205,0.005915,0.001986,0.006091,2.0,0.0,0.0,0.0,0.0
2,성수 서울숲 팝업 건물 통대관 by SHARE.IT,서울 성동구,서울 성동구 서울숲4길 15-1 통건물,성수1가2동,6000000.0,6.0,0.0,697.0,23.0,4182.0,...,0.003625,0.001205,0.005915,0.001986,0.006091,2.0,0.0,0.0,0.0,0.0
3,서울숲 2.7,서울 성동구,서울 성동구 서울숲2길 27,성수1가2동,4000000.0,1.0,1.0,60.0,23.0,60.0,...,0.003625,0.001205,0.005915,0.001986,0.006091,2.0,0.0,0.0,0.0,0.0
4,성수 픽닷 옆 매장 팝업스토어,서울 성동구,서울 성동구 서울숲4길 23-1 102호,성수1가2동,1000000.0,1.0,1.0,26.4,23.0,26.4,...,0.003625,0.001205,0.005915,0.001986,0.006091,2.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
100,매드포갈릭 예술의전당점,서울 서초구,서울 서초구 반포대로 22 서초평화빌딩 지하1층,서초3동,500000.0,1.0,0.0,227.5,1.0,227.5,...,0.008270,0.008394,0.010697,0.004508,0.001354,0.0,0.0,0.0,0.0,0.0
101,이비스 호텔 로비공간,서울 강남구,서울 강남구 삼성로 431,대치4동,0.0,1.0,1.0,1050.0,24.0,1050.0,...,0.005177,0.010916,0.007319,0.001652,0.012816,0.0,0.0,0.0,0.0,0.0
102,더리빙룸at북촌빈관,서울 가회동,서울 종로구 북촌로11가길 10 1층,가회동,228000.0,1.0,1.0,99.0,1.0,99.0,...,0.001078,0.000075,0.002452,0.000337,0.000395,1.0,0.0,0.0,0.0,244.0
103,갤러리 지오타 (한옥),서울 가회동,서울 종로구 북촌로 63-9,가회동,250000.0,1.0,1.0,66.0,10.0,66.0,...,0.001078,0.000075,0.002452,0.000337,0.000395,1.0,0.0,0.0,0.0,244.0


## END OF SCRIPT